# Q1.1

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
import itertools
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq


The set folders should be in the wd. 
The output file will be in the wd.

## Choose for which set you want to produce the parquet file by uncommenting corresponding lines

In [12]:
data_path = os.path.expanduser("set-a/")
file_list = glob.glob(os.path.join(data_path, "*.txt"))  # Get all .txt files
case = 'a'

# data_path = os.path.expanduser("set-b/")
# file_list = glob.glob(os.path.join(data_path, "*.txt"))  
# case = 'b'

# data_path = os.path.expanduser("set-c/")
# file_list = glob.glob(os.path.join(data_path, "*.txt"))
# case = 'c'

len(file_list)

4000

In [13]:

#Question 
# In the Q1.1 paragraph, they say 37 columns for the 37 times series

static_variables = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']
static_variables_we_want = ['Age', 'Gender', 'Height', 'Weight']
all_variables = ['Weight', 'Age', 'TroponinI', 'DiasABP', 'MechVent', 'HCO3', 'Cholesterol', 'HCT', 'SaO2', 'WBC', 'SysABP', 'Urine', 'ICUType', 'Gender', 'ALP', 'Creatinine', 'K', 'AST', 'Glucose', 'RespRate', 'MAP', 'FiO2', 'BUN', 'Na', 'Bilirubin', 'TroponinT', 'PaCO2', 'Height', 'GCS', 'HR', 'pH', 'PaO2', 'Lactate', 'ALT', 'NISysABP', 'RecordID', 'Platelets', 'Temp', 'Mg', 'NIDiasABP', 'Albumin', 'NIMAP']
dyn_variables = [x for x in all_variables if x not in static_variables]
dyn_variables.append('Weight_var')
len(dyn_variables), len(static_variables_we_want)

def time_to_decimal(time_str):
    h, m = map(int, time_str.split(":"))  # Extract hours and minutes
    return h + m / 60  # Convert to decimal

def time_to_hour(time_str):
    h, m = map(int, time_str.split(":"))  # Extract hours and minutes
    return h

In [14]:
#  create empty dataframe
columns_for_df = ['RecordID'] + static_variables_we_want +['Timestamp'] + dyn_variables
len(columns_for_df)

full_df = pd.DataFrame(columns=columns_for_df)
full_df

list_of_patient_dfs = []

In [15]:
# patient_data_df=pd.read_csv(file_list[1])
# patient_data_df['Hour'] = patient_data_df['Time'].apply(time_to_hour)
# patient_data_df


In [ ]:
for k in tqdm(range(len(file_list))): 
    """
    for each patient file, we will create a dataframe with 49 rows, each row represents an hour from 0 to 48

    for each hour row, we have the static variable followed by the dynamic variables average on available values for this hour
    if no value is available for this hour is available for the dynamic variable, we put NaN

    special cases are treated as well
    outliers values are removed 

    for the dynamic variables, we take the mean of the values for this hour
    execpt for urine, we take the sum of the values for this hour
    execpt for temp, we take the max of the values for this
    
    """

    patient_data_df=pd.read_csv(file_list[k])
    patient_data_df['Hour'] = patient_data_df['Time'].apply(time_to_hour)

    ###########################################################################################################
    #  special cases to be treated -strange values removed

    # Replace Gender if -1 with 1 (assuming gender values should be 1 or 0)
    patient_data_df.loc[
        (patient_data_df['Parameter'] == 'Gender') & (patient_data_df['Value'] < 0), 'Value'
    ] = 1

    # Replace Height with NaN if out of (100, 250)
    patient_data_df.loc[
        (patient_data_df['Parameter'] == 'Height') & 
        ((patient_data_df['Value'] < 100) | (patient_data_df['Value'] > 250)), 'Value'
    ] = np.nan

    # Replace Weight with NaN if < 20
    patient_data_df.loc[
        (patient_data_df['Parameter'] == 'Weight') & (patient_data_df['Value'] < 20), 'Value'
    ] = np.nan

    # Replace HR with NaN if < 10 (for multiple rows)
    patient_data_df.loc[
        (patient_data_df['Parameter'] == 'HR') & (patient_data_df['Value'] < 10), 'Value'
    ] = np.nan

    for c in ["DiasABP", "MAP", "NIDiasABP", "NIMAP", "NISysABP", "SysABP", "RespRate", "PaO2"]: 
        # replace with NaN if <= 0
        patient_data_df.loc[
            (patient_data_df['Parameter'] == c) & (patient_data_df['Value'] <= 0), 'Value'
        ] = np.nan

    # replace pH with NaN if < 6 or > 8
    patient_data_df.loc[
        (patient_data_df['Parameter'] == 'pH') & ((patient_data_df['Value'] < 6) | (patient_data_df['Value'] > 8)), 'Value'
    ] = np.nan

    # replace Temp with NaN if < 12 or > 45
    patient_data_df.loc[
        (patient_data_df['Parameter'] == 'Temp') & ((patient_data_df['Value'] < 12) | (patient_data_df['Value'] > 45)), 'Value'
    ] = np.nan
    created_patient_df = pd.DataFrame(columns=columns_for_df)

    ###########################################################################################################

    r_id = patient_data_df[patient_data_df['Parameter']=='RecordID']['Value'].iloc[0]
    age = patient_data_df[patient_data_df['Parameter']=='Age']['Value'].iloc[0]
    height = patient_data_df[patient_data_df['Parameter']=='Height']['Value'].iloc[0]
    weight = patient_data_df[patient_data_df['Parameter']=='Weight']['Value'].iloc[0] # first weight value
    gender = patient_data_df[patient_data_df['Parameter']=='Gender']['Value'].iloc[0]
    static_values = [r_id, age, gender, height, weight ] # with id 
    
    #  remove the first 5 rows in the patient_data_df
    patient_data_df = patient_data_df.iloc[6:]
    patient_data_df

    # add first row in created_patient_df
    created_patient_df.loc[len(created_patient_df)] = static_values + [0] + [np.nan]*len(dyn_variables)



    # create 49 rows for each hour
    for i in range(48): # i is the hour col value we consider , and we note it to the next hour
        static_and_time_values = static_values + [i+1]

        rows_i = patient_data_df[patient_data_df['Hour']==i]
        rows_i = rows_i.groupby('Parameter')['Value'].mean()

        dyn_var_mean = [rows_i[dyn_var] if dyn_var in rows_i.index else np.nan for dyn_var in dyn_variables]

        if ('Weight' in rows_i.index):
            dyn_var_mean[-1] = rows_i['Weight']
        
        # urine case 
        if ('Urine' in rows_i.index): # sum instead of mean
            index_of_urine = dyn_variables.index('Urine')
            dyn_var_mean[index_of_urine] = patient_data_df[patient_data_df['Hour']==i].groupby('Parameter')['Value'].sum()['Urine']

        if ('Temp' in rows_i.index): # max instead of mean
            index_of_temp = dyn_variables.index('Temp')
            dyn_var_mean[index_of_temp] = patient_data_df[patient_data_df['Hour']==i].groupby('Parameter')['Value'].max()['Temp']
        
        
        created_patient_df.loc[len(created_patient_df)] = static_and_time_values + dyn_var_mean
    
    list_of_patient_dfs.append(created_patient_df)
    


  0%|          | 0/4000 [00:00<?, ?it/s]

100%|██████████| 4000/4000 [12:07<00:00,  5.50it/s]


In [17]:
"""
in the end we concatenate all the patients dataframes (one dataframe has been created for each patient)

then we save the final dataframe in a parquet file
"""
final_df = pd.concat(list_of_patient_dfs, ignore_index=True)


In [18]:
final_df

,RecordID,Age,Gender,Height,Weight,Timestamp,TroponinI,DiasABP,MechVent,HCO3,...,Lactate,ALT,NISysABP,Platelets,Temp,Mg,NIDiasABP,Albumin,NIMAP,Weight_var
0,132539.0,54.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,132539.0,54.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,152.0,NaN,35.6,NaN,61.5,NaN,91.665,NaN
2,132539.0,54.0,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,137.0,NaN,NaN,NaN,62.0,NaN,87.000,NaN
3,132539.0,54.0,0.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,123.0,NaN,NaN,NaN,52.0,NaN,75.670,NaN
4,132539.0,54.0,0.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,114.0,NaN,37.8,NaN,52.0,NaN,72.670,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195995,142673.0,78.0,0.0,157.5,70.7,44.0,NaN,52.0,NaN,NaN,...,NaN,NaN,122.0,NaN,37.7,NaN,87.0,NaN,90.000,87.3
195996,142673.0,78.0,0.0,157.5,70.7,45.0,NaN,49.0,NaN,NaN,...,NaN,NaN,101.0,NaN,37.6,NaN,55.0,NaN,63.000,87.3
195997,142673.0,78.0,0.0,157.5,70.7,46.0,NaN,50.0,NaN,NaN,...,NaN,NaN,120.0,NaN,NaN,NaN,82.0,NaN,92.000,87.3
195998,142673.0,78.0,0.0,157.5,70.7,47.0,NaN,54.0,NaN,NaN,...,NaN,NaN,133.0,NaN,37.3,NaN,67.0,NaN,83.000,87.3


In [19]:
converted_table = pa.Table.from_pandas(final_df)

In [20]:
if (case == 'a'):
    pq.write_table(converted_table, 'set-a.parquet')
elif (case == 'b'):
    pq.write_table(converted_table, 'set-b.parquet')
else:
    pq.write_table(converted_table, 'set-c.parquet')

## Labels - outcomes file processing, uncomment if you want to run it

In [21]:
# # open outcomes-a.txt
# outcomes = pd.read_csv('outcomes-a.txt')
# outcomes

# # open outcomes-b.txt and concatenate it to outcomes
# outcomes_b = pd.read_csv('outcomes-b.txt')
# outcomes = pd.concat([outcomes, outcomes_b], ignore_index=True)

# # open outcomes-c.txt and concatenate it to outcomes
# outcomes_c = pd.read_csv('outcomes-c.txt')
# outcomes = pd.concat([outcomes, outcomes_c], ignore_index=True)

# # outcomes : drop columns : SAPS-I	SOFA	Length_of_stay	Survival
# outcomes = outcomes.drop(['SAPS-I', 'SOFA', 'Length_of_stay', 'Survival'], axis=1)
# outcomes

# # save outcomes to csv file
# outcomes.to_csv('outcomes.csv', index=False)